# Inference from saved model TF2


##Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data
!pip install -q object_detection

%cd /content/drive/My Drive/projects/ML/object_detection_tf/workspace

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/projects/ML/object_detection_tf/workspace


In [3]:
import io
import os
import random
import scipy.misc
import numpy as np
import six
import time


from six import BytesIO

import matplotlib.pyplot as plt
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "y",
          "axes.facecolor" : "None",
          "text.color" : 'w'
}
plt.rcParams.update(params)

from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline



## Model Selection

In [4]:
selected_model = "faster_rcnn_resnet101_v1_800x1333" #@param ["ssd_mobilenet_v2_320x320", "ssd_mobilenet_V2_fpnlite_320x320", "ssd_mobilenet_V2_fpnlite_640x640", "ssdlite_mobilenet_edgetpu_coco_quant", "faster_rcnn_resnet101_v1_800x1333"]
model_dir = f'/content/drive/My Drive/projects/ML/object_detection_tf/workspace/exported_models/{selected_model}/saved_model/'

In [5]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Load the COCO Label Map
category_index = {
    1: {'id': 1, 'name': 'coffee'},
    # 2: {'id': 2, 'name': 'good'},
}

In [6]:
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load(model_dir)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed time: {int(elapsed_time)}s')

Elapsed time: 16s


In [9]:
IMAGES_DIRECTORY = 'test_images/old_juan/'
MIN_SCORE_THRESHOLD = 0.3
elapsed = []

NUM_IMAGES = len(os.listdir(IMAGES_DIRECTORY))
for filename in os.listdir(IMAGES_DIRECTORY):
# NUM_IMAGES = 5
# for i in range(NUM_IMAGES): #for random files
#   filename = random.choice(os.listdir(IMAGES_DIRECTORY))
  if os.path.splitext(filename)[1]!='.jpg':
    continue
  image_np = load_image_into_numpy_array(IMAGES_DIRECTORY + filename)
  
  input_tensor = np.expand_dims(image_np, 0)

  start_time = time.time()
  detections = detect_fn(input_tensor)
  end_time = time.time()
  
  boxes=detections['detection_boxes'][0].numpy()
  classes=detections['detection_classes'][0].numpy().astype(np.int32)
  scores=detections['detection_scores'][0].numpy()
  number_detections = np.sum(detections["detection_scores"][0].numpy() > MIN_SCORE_THRESHOLD)

  elapsed.append(end_time - start_time)
  image_np_with_detections = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image=image_np_with_detections,
        boxes=boxes,
        classes=classes,
        scores=scores,
        category_index=category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=1800,
        min_score_thresh=MIN_SCORE_THRESHOLD,
        agnostic_mode=False)
  
  bad_detections = np.sum(classes[scores>MIN_SCORE_THRESHOLD]==1)
  good_detections = np.sum(classes[scores>MIN_SCORE_THRESHOLD]==2)
  total_detections = np.sum(scores>MIN_SCORE_THRESHOLD)
  overlaped_detections = - (total_detections - good_detections - bad_detections)

  number_detections = np.sum(detections["detection_scores"][0].numpy() > MIN_SCORE_THRESHOLD)
  image = Image.fromarray(image_np_with_detections)
  image.save(f'test_images/{os.path.splitext(filename)[0]}_G{good_detections}_B{bad_detections}.jpg')

  plt.figure(figsize = (100, 100))
  plt.title(f'{filename}: boxes: {number_detections}, good_detections: {good_detections}, bad_detections: {bad_detections}, overlaped_detections: {overlaped_detections}')
  plt.imshow(image_np_with_detections)
  
mean_elapsed = sum(elapsed) / float(len(elapsed))
print(f'Elapsed time: {mean_elapsed:.1f}s per image')

Elapsed time: 32.4s per image
